In [ ]:
# brew install graphviz
# pip install graphviz
from graphviz import Digraph

In [ ]:
from micrograd.engine import Value

In [ ]:
def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})

    # Set background color for the entire graph
    dot.graph_attr['bgcolor'] = '#282C35'  # Dark background

    # Set default attributes for nodes
    dot.node_attr['fontcolor'] = '#e6e6e6'  # Light font color
    dot.node_attr['style'] = 'filled'
    dot.node_attr['color'] = '#e6e6e6'  # Light border color
    dot.node_attr['fillcolor'] = '#333333' # Dark fill color for nodes

    # Set default attributes for edges
    dot.edge_attr['color'] = '#e6e6e6'  # Light edge color
    dot.edge_attr['fontcolor'] = '#e6e6e6' # Light font color for edge labels
    
    for n in nodes:
        # dot.node(name=str(id(n)), label = "{ %s | %.2f | ∇ %.3f }" % (n.label, n.data, n.grad), shape='record')
        minus = '−'  # Unicode U+2212
        dot.node(name=str(id(n)), label="{ %s | %s | ∇: %s }" % (
            n.label,
            f"{n.data:.2f}".replace('-', minus),
            f"{n.grad:.3f}".replace('-', minus)
        ), shape='record', color=n.color)
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op, shape='cds', color='aqua')
            dot.edge(str(id(n)) + n._op, str(id(n)))
    
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot

In [ ]:
# a very simple example
x = Value(1.0, label='x')
y = (x * 2 + 1).relu(); y.label = 'y'
y.backward()
draw_dot(y)

In [ ]:
# Leaves
x1  = Value(+1, label='x1')
x2  = Value(+0, label='x2')
w11 = Value(-2, label='w11')
w12 = Value(+4, label='w12')
w21 = Value(+3, label='w21')
w22 = Value(-1, label='w22')
w1  = Value(-1, label='w1')
w2  = Value(1/12,label='w2')
y   = Value(+1, label='y')

# s = W @ x
w11x1 = w11 * x1; w11x1.label = 'w11x1'
w12x2 = w12 * x2; w12x2.label = 'w12x2'
s1 = w11x1 + w12x2; s1.label = 's1'
w21x1 = w21 * x1; w21x1.label = 'w21x1'
w22x2 = w22 * x2; w22x2.label = 'w22x2'
s2 = w21x1 + w22x2; s2.label = 's2'

# h = (s)+
h1 = s1.relu(); h1.label = 'h1'
h2 = s2.relu(); h2.label = 'h2'

# y~ = wTh
w1h1 = w1 * h1; w1h1.label = 'w1h1'
w2h2 = w2 * h2; w2h2.label = 'w2h2'
ytld = w1h1 + w2h2; ytld.label = 'ytld'

# C = (y - ytld)^2
_ytld = -ytld; _ytld.label = '−ytld'
C = (y + _ytld)**2; C.label = 'C'; C.color = 'red'

In [ ]:
draw_dot(C)

In [ ]:
C.backward()

In [ ]:
draw_dot(C)

In [ ]:
# a simple 2D neuron
import random
from micrograd import nn

random.seed(1337)
n = nn.Neuron(2)
x = [Value(1.0), Value(-2.0)]
y = n(x)
y.backward()

dot = draw_dot(y)
dot

In [ ]:
dot.render('gout')